In [2]:
from transformers import AutoTokenizer, RobertaModel
import torch
from torch import nn , optim 
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
import pandas as pd
import lightning as L
from pytorch_lightning.loggers import WandbLogger

/home2/jainit/miniconda3/envs/torchy/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base" , truncation=True, max_length=512 )
model = RobertaModel.from_pretrained("roberta-base" , output_hidden_states=True)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [5]:

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs[2]

In [6]:
# sum first 4 layers
avg_hidden_states = torch.mean(torch.stack(last_hidden_states[-4:]), dim=0)

In [7]:
avg_hidden_states.shape

torch.Size([1, 8, 768])

In [15]:
roberta = RobertaModel.from_pretrained("roberta-base", output_hidden_states=True)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
class LSTM_RoBERTA(nn.Module):
    def __init__(self,roberta_path = "roberta-base",lstm_hidden_size=256,lstm_layers=2,lstm_dropout=0.2 , num_classes=2):
        super().__init__()
        self.roberta = RobertaModel.from_pretrained(roberta_path)
        self.lstm = nn.LSTM(self.roberta.config.hidden_size,lstm_hidden_size,lstm_layers,batch_first=True,dropout=lstm_dropout,bidirectional=True)
        self.linear = nn.Linear(lstm_hidden_size*2,num_classes)
        self.dropout = nn.Dropout(0.2)
        for param in self.roberta.parameters():
            param.requires_grad = False
        for params in self.roberta.pooler.parameters():
            params.requires_grad = True
        print("Model Loaded")
        print("Total_trainable_params : ",sum(p.numel() for p in self.parameters() if p.requires_grad))
        print("Total_untrainable_params : ",sum(p.numel() for p in self.parameters() if not p.requires_grad))
    def forward(self, input_ids, attention_mask):
        roberta_output = self.roberta(input_ids, attention_mask)
        roberta_output = roberta_output.last_hidden_state
        lstm_output, (h_n,c_n) = self.lstm(roberta_output)
        logits = self.linear(lstm_output[:, -1, :])
        # print(logits.shape,"logits")
        return logits

In [6]:
class DatasetLM(torch.utils.data.Dataset):
    def __init__(self, dataset, tokeniser):
        self.dataset = dataset
        self.tokeniser = tokeniser
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, index):
        # print(self.tokeniser(self.dataset[index]['text'], return_tensors="pt")['input_ids'])
        return (self.tokeniser(self.dataset[index]['text'], return_tensors="pt", truncation=True, max_length=512), self.dataset[index]['label'])



In [7]:
train_data_path = "/scratch/jainit/SubtaskA/subtaskA_train_monolingual.jsonl"
test_data_path = "/scratch/jainit/SubtaskA/subtaskA_dev_monolingual.jsonl"
df_train = pd.read_json(train_data_path, lines=True)
df_test = pd.read_json(test_data_path, lines=True)

In [8]:
train_data = df_train.to_dict('records')
test_data = df_test.to_dict('records')

In [9]:
train_dataset = DatasetLM(train_data, tokenizer)
test_dataset = DatasetLM(test_data, tokenizer)

In [10]:
def collate_fn(batch):
    input_ids = [x[0]['input_ids'].squeeze(0) for x in batch]
    # print(input_ids)
    attention_mask = [x[0]['attention_mask'].squeeze(0) for x in batch]
    labels = [x[1] for x in batch]
    # print(batch)
    # input_ids = torch.stack(input_ids)
    # attention_mask = torch.stack(attention_mask)
    # print(input_ids.shape)
    # print([x.shape for x in input_ids])
    inputs_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=1)
    attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)
    # print(inputs_ids.shape)
    x = {'input_ids': inputs_ids, 'attention_mask': attention_mask}
    y = torch.Tensor(labels)
    return (x, y)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

In [12]:
from lightning.pytorch.utilities.types import TRAIN_DATALOADERS


class LitModel(L.LightningModule):
    def __init__(self,train_loader , val_loader , model_path, lr=2e-5, hidden_size=256, lstm_layers=2, lstm_dropout=0.2, num_classes=2):
        super().__init__()
        self.model_path = model_path
        self.lr = lr
        self.hidden_size = hidden_size
        self.lstm_layers = lstm_layers
        self.lstm_dropout = lstm_dropout
        self.num_classes = num_classes
        self.model = LSTM_RoBERTA(model_path, hidden_size, lstm_layers, lstm_dropout, num_classes)
        self.train_loader = train_loader
        self.val_loader = val_loader
        # self.save_hyperparameters()
    
    def forward(self, input_ids, attention_mask):
        # print(input_ids.shape , attention_mask.shape , "Here are the shapes")
        out = self.model(input_ids, attention_mask)
        # print(out.shape , "Here is the output$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
        return out
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        # print(x, y )
        # print( y.shape, "Here are wasd############" )
        logits = self.forward(x['input_ids'], x['attention_mask'])
        # print(logits.shape , "Here are the logits")
        loss = F.cross_entropy(logits, y.long())
        # print(logits.shape , y.shape)
        
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
        # print("##############")
        # return None
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x['input_ids'], x['attention_mask'])
        loss = F.cross_entropy(logits, y.long())
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        acc = (logits.argmax(dim=-1) == y).float().mean()
        self.log('val_acc', acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        
        return loss
    
    def train_dataloader(self):
        return self.train_loader
    
    def val_dataloader(self):
        return self.val_loader
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr , weight_decay=0.01 , eps=1e-6)
        

In [16]:
trainer = L.Trainer(max_epochs=10, devices="auto",val_check_interval=20, logger=WandbLogger(project="roberta-lstm", log_model=False, name="TEST" ))

/home2/jainit/miniconda3/envs/torchy/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home2/jainit/miniconda3/envs/torchy/lib/python3.11/ ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [17]:
trainer.fit(LitModel(train_loader, test_loader, "roberta-base"))